# Document Q&A 

In [ ]:
# ! pip install langchain
# ! pip install openai

# ! pip install pypdf 
# ! pip install langchain-community
# ! pip install sentence-transformers

In [3]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('HF_API_KEY')
# openai.api_key  = os.environ['OPENAI_API_KEY']

In [4]:
from langchain.document_loaders import PyPDFLoader
file_path = "samplePDF.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load()

In [5]:
len(pages)

25

In [6]:
page = pages[0]

In [ ]:
print(page.page_content[0:1000])

In [ ]:
page.metadata

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
# Step 2: Create embeddings and vector store
embeddings = HuggingFaceEmbeddings()  # Use Hugging Face embeddings
vectorstore = DocArrayInMemorySearch.from_documents(pages, embeddings)

# Step 3: function to query the PDF
def query_pdf(query, top_k=3):
    """
    Search the PDF for relevant content based on the query.
    :param query: Your search query (e.g., "What models are used?")
    :param top_k: Number of top results to return
    :return: List of relevant document snippets
    """
    # Perform similarity search
    results = vectorstore.similarity_search(query, k=top_k)
    
    # Display the results
    for i, result in enumerate(results):
        print(f"Result {i + 1}:")
        print(result.page_content)  # Print the content of the relevant page
        print("-" * 80)  # Separator

# Step 4: Query the PDF
query = "what Deep Learning model performed best?"  # Replace with yours
query_pdf(query)

C:\Users\saraja\AppData\Local\Temp\ipykernel_10524\1248922.py:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()  # Use Hugging Face embeddings


Result 1:
hidden layers to output. They are well-suited for tasks where sequential 
order is less relevant. CNNs also use the same structure as FFNN but they 
also include convolutional layers before hidden neurons that process 
fixed-size local receptive fields with automatic feature extraction, 
making them more suitable for grid-like data, e.g., images.
Recurrent neural networks (RNNs) ( Elman, 1990 ; Rumelhart et al., 
1986 ; Werbos, 1988 ) are specialized DL models for tasks involving dy -
namic temporal dependencies, such as natural language processing or 
time series forecasting ( Zou et al., 2023 ). RNNs have found extensive 
applications in various forecasting tasks. Compared to other models, 
RNNs are favored for their memory to handle time series tasks. Another 
distinguishing characteristic of RNNs is that they share parameters 
across each layer of the network, allowing the model to capture tem -
poral patterns and dependencies. Compared to FFNNs which often as -
sume inpu

### improvements with larger huggingface model

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub


# Step 2: Split the text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Split text into chunks of 1000 characters
    chunk_overlap=200,  # Add overlap to avoid losing context
)
chunks = text_splitter.split_documents(pages)

# Step 3: Create embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")  # Use a better embedding model
vectorstore = DocArrayInMemorySearch.from_documents(chunks, embeddings)

_ = load_dotenv(find_dotenv()) # read local .env file
# Access the Hugging Face API key
hf_api_key = os.getenv('HF_API_KEY')

# Step 4: Set up the retrieval-based QA chain
llm = HuggingFaceHub(
    repo_id="google/flan-t5-base",  # Using a more powerful Hugging Face model
    huggingfacehub_api_token=hf_api_key,  # Replace with your Hugging Face API key
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Use "stuff" for small documents
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),  # Retrieve top 5 chunks
)

# Step 5: Query the PDF sample which is an article
# query = "What models are used in this study?"  # Replace with yours
query = "what Deep Learning model performed best?"  # Replace with yours
# query = "what are the deep learning model input variables?"  # Replace with yours
# query = "what is the aim of this study?"  # Replace with yours
response = qa_chain.run(query)

# Step 6: Display the response
print("Response:", response)

Response: GRU


f:\HACKERRANK\LLM\llmvenv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
